# BIDS-Matlab: basics

1. [Indexing a dataset](#Indexing-a-dataset)
1. [Querying a dataset](#Querying-a-dataset)
  1. [Get filenames](#Get-filenames)
  1. [Get metadata](#Get-metadata)
  1. [Get "dependencies" of a given file](#Get-"dependencies"-of-a-given-file)  
1.  [Read from TSV files](#Read-from-TSV-files)
 

In [4]:
% add bids-matlab to path
addpath(fullfile(pwd, '..'));

% TO DO: rename the bids.internal.warning function instead of silencing warnings
warning('off','all');

We will work with the "empty" dataset from the bids-examples repository : https://github.com/bids-standard/bids-examples

We use a bit of command line magic to view the top (`head`) directories (`-d`) at a certain level depth (`-L 2`).

In [38]:
!tree bids-examples -d -L 2 | head

bids-examples
├── 7t_trt
│   ├── sub-01
│   ├── sub-02
│   ├── sub-03
│   ├── sub-04
│   ├── sub-05
│   ├── sub-06
│   ├── sub-07
│   ├── sub-08



Let's work on the `ds007` dataset.

In [39]:
!tree bids-examples/ds007 -d -L 2 | head

bids-examples/ds007
├── sub-01
│   ├── anat
│   └── func
├── sub-02
│   ├── anat
│   └── func
├── sub-03
│   ├── anat
│   └── func



Parse BIDS directory

## Indexing a dataset

This is done with the `bids.layout` function.

In [36]:
help bids.layout



 Parse a directory structure formated according to the BIDS standard

 USAGE::

   BIDS = bids.layout(root = pwd, use_schema = false)

 :param root:       directory of the dataset formated according to BIDS [default: ``pwd``]
 :type  root:       string
 :param use_schema: If set to ``true`` (default), the parsing of the dataset
                    will follow the bids-schema provided with bids-matlab.
                    If set to ``false`` files just have to be of the form
                    ``sub-label_[entity-label]_suffix.ext`` to be parsed.
                    If a folder path is provided, then the schema contained
                    in that folder willl be used for parsing.
 :type  use_schema: boolean


 (C) Copyright 2016-2018 Guillaume Flandin, Wellcome Centre for Human Neuroimaging

 (C) Copyright 2018 BIDS-MATLAB developers


Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the 

In [35]:
BIDS = bids.layout(fullfile(pwd,'bids-examples','ds007'));

## Querying a dataset

Make general queries about the dataset are with `bids.query` made on the layout returned by `bids.layout`.

In [37]:
help bids.query



 Queries a directory structure formatted according to the BIDS standard

 USAGE::

   result = bids.query(BIDS, query, ...)

 :param BIDS: BIDS directory name or BIDS structure (from bids.layout)
 :type  BIDS: strcuture or string
 :param query: type of query:
                          - 'data',
                          - 'metadata',
                          - 'sessions',
                          - 'subjects',
                          - 'runs',
                          - 'tasks',
                          - 'suffixes',
                          - 'modalities'
 :type  query: string

 Queries can "filtered" by passing more arguments key-value pairs as a list of
 strings or as a cell or a structure

 Example 1::

    data = bids.query(BIDS, 'data', ...
                            'sub', '01', ...
                            'task', 'stopsignalwithpseudowordnaming', ...
                            'extension', '.nii.gz', ...
                            'suffix', 'bold');


 Example 2

In [6]:
bids.query(BIDS,'subjects')

ans = 
{
  [1,1] = 01
  [1,2] = 02
  [1,3] = 03
  [1,4] = 04
  [1,5] = 05
  [1,6] = 06
  [1,7] = 07
  [1,8] = 08
  [1,9] = 09
  [1,10] = 10
  [1,11] = 11
  [1,12] = 12
  [1,13] = 13
  [1,14] = 14
  [1,15] = 15
  [1,16] = 16
  [1,17] = 17
  [1,18] = 18
  [1,19] = 19
  [1,20] = 20
}


In [12]:
bids.query(BIDS,'sessions')

ans = {}(1x0)


In [8]:
bids.query(BIDS,'runs')

ans = 
{
  [1,1] = 01
  [1,2] = 02
}


In [13]:
bids.query(BIDS,'tasks')

ans = 
{
  [1,1] = stopsignalwithletternaming
  [1,2] = stopsignalwithmanualresponse
  [1,3] = stopsignalwithpseudowordnaming
}


In [15]:
bids.query(BIDS,'suffixes')

ans = 
{
  [1,1] = T1w
  [1,2] = bold
  [1,3] = events
  [1,4] = inplaneT2
}


In [16]:
bids.query(BIDS,'modalities')

ans = 
{
  [1,1] = anat
  [1,2] = func
}


In [18]:
% Make more specific queries
bids.query(BIDS,'runs', 'suffix', 'T1w')

ans = {}(1x0)


In [19]:
bids.query(BIDS,'runs', 'suffix', 'bold')

ans = 
{
  [1,1] = 01
  [1,2] = 02
}


### Get filenames

Get the NIfTI file for subject `'05'`, run `'02'` and task `'stopsignalwithmanualresponse'`:

In [45]:
bids.query(BIDS,'data','sub','05','run','02','task','stopsignalwithmanualresponse','suffix','bold')

ans = 
{
  [1,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-05/func/sub-05_task-stopsignalwithmanualresponse_run-02_bold.nii.gz
}


This can be also done by creating a structure that can be used as a library.

In [24]:
filter = struct(...
    'sub','05', ...
    'run','02', ...
    'task','stopsignalwithmanualresponse', ...
    'suffix','bold');

filter =

  scalar structure containing the fields:

    sub = 05
    run = 02
    task = stopsignalwithmanualresponse
    suffix = bold



In [25]:
bids.query(BIDS, 'data', filter)

ans = 
{
  [1,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-05/func/sub-05_task-stopsignalwithmanualresponse_run-02_bold.nii.gz
}


You can also query data from several labels or indices

In [31]:
filter.sub = {'01', '03'}

filter =

  scalar structure containing the fields:

    sub = 
    {
      [1,1] = 01
      [1,2] = 03
    }
    run = 02
    task = stopsignalwithmanualresponse
    suffix = bold



In [32]:
bids.query(BIDS, 'data', filter)

ans = 
{
  [1,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-01/func/sub-01_task-stopsignalwithmanualresponse_run-02_bold.nii.gz
  [2,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-03/func/sub-03_task-stopsignalwithmanualresponse_run-02_bold.nii.gz
}


### Get metadata

We can also get the metadata of that file including TR:

In [33]:
bids.query(BIDS, 'metadata', filter)

ans = 
{
  [1,1] =

    scalar structure containing the fields:

      RepetitionTime =  2
      TaskName = stop signal with manual response

  [1,2] =

    scalar structure containing the fields:

      RepetitionTime =  2
      TaskName = stop signal with manual response

}


Get the T1-weighted images from all subjects:

In [34]:
bids.query(BIDS, 'data', 'suffix', 'T1w')

ans = 
{
  [1,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-01/anat/sub-01_T1w.nii.gz
  [2,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-02/anat/sub-02_T1w.nii.gz
  [3,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-03/anat/sub-03_T1w.nii.gz
  [4,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-04/anat/sub-04_T1w.nii.gz
  [5,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-05/anat/sub-05_T1w.nii.gz
  [6,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-06/anat/sub-06_T1w.nii.gz
  [7,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-07/anat/sub-07_T1w.nii.gz
  [8,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-08/anat/sub-08_T1w.nii.gz
  [9,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-09/anat/sub-09_T1w.nii.gz
  [10,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-10/anat/sub-10_T1w.nii.gz


### Get "dependencies" of a given file

This can be useful to find the files that are associated with the file you just queried.

In this case the events file for a BOLD run.

In [41]:
filter = struct(...
    'sub','05', ...
    'run','02', ...
    'task','stopsignalwithmanualresponse', ...
    'suffix','bold');
    
dependencies = bids.query(BIDS, 'dependencies', filter)    

dependencies =

  scalar structure containing the fields:

    explicit = {}(0x0)
    data = {}(0x0)
    group = 
    {
      [1,1] = /home/remi/github/BIDS-matlab/examples/bids-examples/ds007/sub-05/func/sub-05_task-stopsignalwithmanualresponse_run-02_events.tsv
    }



## Read from TSV files

This can be done with the `bids.util.tsvread function`.

In [43]:
tsv_file = dependencies.group{1};

In [44]:
bids.util.tsvread(tsv_file)

ans =

  scalar structure containing the fields:

    onset =

         0.00000
         2.00000
         4.25000
         8.62500
        12.37500
        15.25000
        18.25000
        21.25000
        23.37500
        26.37500
        28.62500
        31.25000
        34.87500
        38.12500
        40.25000
        42.25000
        45.25000
        47.50000
        49.50000
        52.87500
        55.75000
        58.37500
        62.50000
        64.87500
        69.12500
        71.12500
        73.25000
        76.25000
        80.12500
        82.50000
        85.12500
        87.75000
        90.25000
        92.37500
        94.37500
        97.62500
       101.00000
       105.75000
       108.87500
       111.25000
       114.25000
       118.00000
       120.12500
       122.62500
       125.25000
       129.12500
       131.25000
       133.37500
       136.00000
       139.12500
       142.00000
       144.12500
       146.62500
       149.25000
       151.87500
  